In [3]:
import cv2
import numpy as np

def detect_noodle_features(img):
    """More robust noodle detection with adaptive thresholding"""
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    # Adaptive thresholding for light regions
    adaptive_light = cv2.adaptiveThreshold(
        l, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 51, 7)

    # Yellow detection in normalized color space
    mean_b = np.mean(b)
    std_b = np.std(b)
    yellow_mask = cv2.inRange(b, mean_b + std_b, 255)

    combined = cv2.bitwise_or(adaptive_light, yellow_mask)

    """Segment bright/yellow noodle regions and return fraction of image area."""
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Threshold for white noodles (low saturation, high value)
    lower_white = np.array([0, 0, 200]); upper_white = np.array([180, 30, 255])
    mask_white = cv2.inRange(hsv, lower_white, upper_white)
    # Threshold for yellow noodles (Hue ~20-35)
    lower_yellow = np.array([20, 100, 100]); upper_yellow = np.array([35, 255, 255])
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    # Combine and clean up
    mask_noodle = cv2.bitwise_or(mask_white, mask_yellow)
    mask_noodle = cv2.morphologyEx(mask_noodle, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
    # Compute total noodle-like area
    contours, _ = cv2.findContours(mask_noodle, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    noodle_area = sum(cv2.contourArea(c) for c in contours)
    area_score = noodle_area / (img.shape[0] * img.shape[1] + 1e-6)
    return area_score, mask_noodle

def detect_soup_type(img):
    """Sample soup color in the center: return 'light' for Pho, 'red' for Bun Bo, else 'other'."""
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    # Sample central region of image (assuming bowl roughly centered)
    center = (slice(img.shape[0]//4, 3*img.shape[0]//4), slice(img.shape[1]//4, 3*img.shape[1]//4))
    avg_h = np.mean(h[center]); avg_s = np.mean(s[center]); avg_v = np.mean(v[center])
    # Light, low-saturation broth (Pho)
    if avg_v > 150 and avg_s < 80:
        return 'light'
    # Dark red broth (Bun Bo Hue)
    if (avg_h < 15 or avg_h > 165) and avg_s > 100:
        return 'red'
    return 'other'

def detect_pho_bunbo_miquang(img):
    """Assign scores to Pho, Bun Bo Hue, Mi Quang based on noodle and soup features."""
    noodle_score, mask = detect_noodle_features(img)
    scores = {'pho': 0.0, 'bun_bo': 0.0, 'mi_quang': 0.0}
    # If noodles are present, decide based on broth color
    if noodle_score > 0.01:
        soup = detect_soup_type(img)
        base = min(noodle_score * 10, 1.0)
        if soup == 'light':
            scores['pho'] = base
        if soup == 'red':
            scores['bun_bo'] = base
        # Check yellow ratio within noodle mask for Mi Quang
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask_yellow = cv2.inRange(hsv, np.array([20,100,100]), np.array([35,255,255]))
        yellow_count = cv2.countNonZero(mask_yellow & mask)
        white_count = cv2.countNonZero(cv2.inRange(hsv, np.array([0,0,200]), np.array([180,30,255])) & mask)
        if yellow_count + white_count > 0:
            yellow_ratio = yellow_count / (yellow_count + white_count)
            if yellow_ratio > 0.3:
                scores['mi_quang'] = base * yellow_ratio
    return scores

def detect_com_tam(img):
    """Detect Com Tam by rice (white), ribs (brown), and egg."""
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # White rice and egg whites
    mask_white = cv2.inRange(hsv, np.array([0,0,200]), np.array([180,60,255]))
    # Brown pork ribs (dark orange-red)
    mask_brown = cv2.inRange(hsv, np.array([0,100,20]), np.array([15,255,80]))
    # Yellow yolk
    mask_yellow = cv2.inRange(hsv, np.array([20,100,100]), np.array([35,255,255]))
    mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
    mask_brown = cv2.morphologyEx(mask_brown, cv2.MORPH_CLOSE, np.ones((5,5),np.uint8))
    # Area ratios
    area_white = cv2.countNonZero(mask_white) / (img.shape[0]*img.shape[1] + 1e-6)
    area_brown = cv2.countNonZero(mask_brown) / (img.shape[0]*img.shape[1] + 1e-6)
    area_yolk  = cv2.countNonZero(mask_yellow)/ (img.shape[0]*img.shape[1] + 1e-6)
    # Detect eggs via Hough Circles (white circle with yellow center)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1.5, minDist=50,
                               param1=50, param2=30, minRadius=10, maxRadius=100)
    egg_count = len(circles[0]) if circles is not None else 0
    # Score as weighted sum; require at least one egg circle
    score = (0.6*area_white + 0.3*area_brown + 0.1*area_yolk) * (1 if egg_count>0 else 0)
    return {'com_tam': float(min(score, 1.0))}

def detect_goi_cuon(img):
    """Detect Goi Cuon by shrimp (red shapes) and cylindrical roll contours."""
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Threshold for shrimp pink/red (split in two hue ranges)
    mask_red1 = cv2.inRange(hsv, np.array([0,120,50]), np.array([10,255,255]))
    mask_red2 = cv2.inRange(hsv, np.array([160,120,50]), np.array([180,255,255]))
    mask_shrimp = cv2.bitwise_or(mask_red1, mask_red2)
    # Count shrimp-like contours
    cnts, _ = cv2.findContours(mask_shrimp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    shrimp_count = sum(1 for c in cnts if cv2.contourArea(c) > 100)
    # Detect elongated roll shapes via edges
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    cnts2, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    roll_count = 0
    for c in cnts2:
        x,y,w,h = cv2.boundingRect(c)
        if w>0 and h>0:
            aspect = max(w,h) / min(w,h)
            if 1.5 < aspect < 5.0 and cv2.contourArea(c) > 500:
                roll_count += 1
    # Combine counts into score (capped at 1.0)
    score = (0.5*shrimp_count + 0.5*roll_count) / 5.0
    return {'goi_cuon': float(min(score,1.0))}

def detect_banh_xeo(img):
    """Detect Banh Xeo by large yellow region and green foliage."""
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Yellow crepe
    mask_yellow = cv2.inRange(hsv, np.array([20,100,100]), np.array([35,255,255]))
    cnts, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    yellow_area = sum(cv2.contourArea(c) for c in cnts if cv2.contourArea(c) > 1000)
    # Green lettuce
    mask_green = cv2.inRange(hsv, np.array([40,50,50]), np.array([80,255,255]))
    green_area = cv2.countNonZero(mask_green)
    score = 0.7*(yellow_area/(img.shape[0]*img.shape[1]+1e-6)) + 0.3*(green_area/(img.shape[0]*img.shape[1]+1e-6))
    return {'banh_xeo': float(min(score,1.0))}

def classify_dish(img):
    """Run all detectors and choose dish with highest confidence score."""
    scores = {'pho':0, 'bun_bo':0, 'mi_quang':0, 'com_tam':0, 'goi_cuon':0, 'banh_xeo':0}
    scores.update(detect_pho_bunbo_miquang(img))
    scores.update(detect_com_tam(img))
    scores.update(detect_goi_cuon(img))
    scores.update(detect_banh_xeo(img))
    # Identify best match
    best = max(scores, key=scores.get)
    return scores, (best, scores[best])

# Example usage:
# img = cv2.imread('dish.jpg')
# scores, (pred, conf) = classify_dish(img)
# print(f"Prediction: {pred} (score {conf:.2f})")

# usage for folder dataset/test/pho
import os
import glob
folder = 'dataset/test/goi cuon'
for filename in glob.glob(os.path.join(folder, '*.jpg')):
    img = cv2.imread(filename)
    scores, (pred, conf) = classify_dish(img)
    print(f"File: {filename}, Prediction: {pred} (score {conf:.2f})")


File: dataset/test/goi cuon/823.jpg, Prediction: pho (score 1.00)
File: dataset/test/goi cuon/610.jpg, Prediction: goi_cuon (score 1.00)
File: dataset/test/goi cuon/348.jpg, Prediction: goi_cuon (score 1.00)
File: dataset/test/goi cuon/360.jpg, Prediction: pho (score 1.00)
File: dataset/test/goi cuon/758.jpg, Prediction: goi_cuon (score 0.50)
File: dataset/test/goi cuon/770.jpg, Prediction: goi_cuon (score 1.00)
File: dataset/test/goi cuon/764.jpg, Prediction: goi_cuon (score 1.00)
File: dataset/test/goi cuon/765.jpg, Prediction: goi_cuon (score 1.00)
File: dataset/test/goi cuon/413.jpg, Prediction: mi_quang (score 0.27)
File: dataset/test/goi cuon/361.jpg, Prediction: goi_cuon (score 0.50)
File: dataset/test/goi cuon/89.jpg, Prediction: goi_cuon (score 0.70)
File: dataset/test/goi cuon/822.jpg, Prediction: pho (score 0.51)
File: dataset/test/goi cuon/149.jpg, Prediction: pho (score 1.00)
File: dataset/test/goi cuon/405.jpg, Prediction: goi_cuon (score 1.00)
File: dataset/test/goi cuon

KeyboardInterrupt: 